In [1]:
!apt-get update
!apt-get install -y libgl1-mesa-glx

!pip install h5py
!pip install scikit-learn
!pip install opencv-python

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:2 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Reading package lists... Done
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libdrm-amdgpu1 libdrm-common libdrm-intel1 libdrm-nouveau2 libdrm-radeon1
  libdrm2 libgl1 libgl1-amber-dri libgl1-mesa-dri libglapi-mesa libglvnd0
  libglx-mesa0 libglx0 libllvm15 libpciaccess0 libsensors-config libsensors5
  libx11-6 libx11-data libx11-xcb1 libxau6 libxcb-dri2-0 libxcb-dri3-0
  libxcb-glx0 libxcb-present0 libxcb-randr0 libxcb-shm0 libxcb-sync1
  libxcb-xfixes0 libxcb1 libxdmcp6 libxext6 libxfixes3 libxshmfence1
  libxxf86vm1
Suggested packages:
  pciutils lm-sensors
The following NEW packages will be installed:
  libdrm-amdgpu1 libdrm-common lib

In [2]:
import h5py
import torch
from torch.utils.data import Dataset, DataLoader
from torch import optim
from tqdm import tqdm
from torchvision.transforms import ToTensor, ToPILImage
from torchvision.utils import save_image
from PIL import Image
import os
import numpy as np
from sklearn.model_selection import train_test_split

from torchvision import transforms

class RainDataset(Dataset):
    def __init__(self, input_paths, target_paths):
        super().__init__()
        self.input_paths = input_paths
        self.target_paths = target_paths
        self.transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Resize((256, 256)),
            transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
        ])
        
    def __len__(self):
        return len(self.input_paths)
    
    def __getitem__(self, idx):
        input_image = Image.open(self.input_paths[idx]).convert("RGB")
        target_image = Image.open(self.target_paths[idx]).convert("RGB")
        
        input_image = self.transform(input_image)
        target_image = self.transform(target_image)
        return input_image, target_image

In [3]:

import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.utils.data as Data
import torch.nn.functional as F
import torchvision

import numpy as np
import cv2
import random
import time
import os

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable


class PReNet(nn.Module):
    def __init__(self, recurrent_iter=6, use_GPU=True):
        super(PReNet, self).__init__()
        self.iteration = recurrent_iter
        self.use_GPU = use_GPU

        self.conv0 = nn.Sequential(
            nn.Conv2d(6, 16, 3, 1, 1), 
            nn.ReLU()
        )
        
        self.res_conv1 = nn.Sequential(
            nn.Conv2d(16, 16, 3, 1, 1),
            nn.ReLU(),
            nn.Conv2d(16, 16, 3, 1, 1),
            nn.ReLU()
        )
        
        self.res_conv2 = nn.Sequential(
            nn.Conv2d(16, 16, 3, 1, 1),
            nn.ReLU(),
            nn.Conv2d(16, 16, 3, 1, 1),
            nn.ReLU()
        )
        
        self.conv_i = nn.Sequential(
            nn.Conv2d(16 + 16, 16, 3, 1, 1),
            nn.Sigmoid()
        )
        self.conv_f = nn.Sequential(
            nn.Conv2d(16 + 16, 16, 3, 1, 1),
            nn.Sigmoid()
        )
        self.conv_g = nn.Sequential(
            nn.Conv2d(16 + 16, 16, 3, 1, 1),
            nn.Tanh()
        )
        self.conv_o = nn.Sequential(
            nn.Conv2d(16 + 16, 16, 3, 1, 1),
            nn.Sigmoid()
        )
        
        self.conv = nn.Sequential(
            nn.Conv2d(16, 3, 3, 1, 1),
        )

    def forward(self, input):
        batch_size, row, col = input.size(0), input.size(2), input.size(3)

        x = input
        h = Variable(torch.zeros(batch_size, 16, row, col))
        c = Variable(torch.zeros(batch_size, 16, row, col))

        if self.use_GPU:
            h = h.cuda()
            c = c.cuda()

        x_list = []
        for i in range(self.iteration):
            x = torch.cat((input, x), 1)
            x = self.conv0(x)

            x = torch.cat((x, h), 1)
            i = self.conv_i(x)
            f = self.conv_f(x)
            g = self.conv_g(x)
            o = self.conv_o(x)
            c = f * c + i * g
            h = o * torch.tanh(c)

            x = h
            resx = x
            x = F.relu(self.res_conv1(x) + resx)
            resx = x
            x = F.relu(self.res_conv2(x) + resx)
            
            x = self.conv(x)

            x = x + input
            x_list.append(x)

        return x, x_list


In [3]:
def l1_loss(pred, target):
    return torch.mean(torch.abs(pred - target))

In [4]:
# ============================
# Helper functions
# ============================
def psnr(x, y, data_range=1.0): 
    mse = torch.mean((x - y) ** 2)
    if mse == 0:
        return float('inf')
    return 20 * torch.log10(data_range / torch.sqrt(mse))


def ssim(x, y, kernel_size=11, kernel_sigma=1.5, data_range=1.0, k1=0.01, k2=0.03):
    coords = torch.arange(kernel_size, dtype=x.dtype, device=x.device)
    coords -= (kernel_size - 1) / 2.0
    g = coords ** 2
    g = (- (g.unsqueeze(0) + g.unsqueeze(1)) / (2 * kernel_sigma ** 2)).exp()
    g /= g.sum()
    kernel = g.unsqueeze(0).repeat(x.size(1), 1, 1, 1)

    c1, c2 = (k1 * data_range) ** 2, (k2 * data_range) ** 2
    n_channels = x.size(1)
    mu_x = F.conv2d(x, weight=kernel, stride=1, padding=0, groups=n_channels)
    mu_y = F.conv2d(y, weight=kernel, stride=1, padding=0, groups=n_channels)

    mu_xx, mu_yy, mu_xy = mu_x ** 2, mu_y ** 2, mu_x * mu_y
    sigma_xx = F.conv2d(x ** 2, weight=kernel, stride=1, padding=0, groups=n_channels) - mu_xx
    sigma_yy = F.conv2d(y ** 2, weight=kernel, stride=1, padding=0, groups=n_channels) - mu_yy
    sigma_xy = F.conv2d(x * y, weight=kernel, stride=1, padding=0, groups=n_channels) - mu_xy

    cs = (2.0 * sigma_xy + c2) / (sigma_xx + sigma_yy + c2)
    ss = (2.0 * mu_xy + c1) / (mu_xx + mu_yy + c1) * cs
    return ss.mean()

In [5]:
def train(model, dataloader, optimizer, device):
    model.train()
    running_loss = 0.0
    l1_loss_fn = torch.nn.L1Loss()
    
    for inputs, targets in tqdm(dataloader, desc="Training"):
        inputs, targets = inputs.to(device), targets.to(device)
        
        optimizer.zero_grad()
        output_images, _ = model(inputs)  
        
        loss = l1_loss_fn(output_images, targets)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    avg_loss = running_loss / len(train_loader)
    return avg_loss

def evaluate(model, dataloader, device):
    model.eval()
    total_l1_loss = 0.0
    total_psnr = 0.0
    total_ssim = 0.0
    l1_loss_fn = torch.nn.L1Loss()
    
    with torch.no_grad():
        for inputs, targets in tqdm(dataloader, desc="Evaluating"):
            inputs, targets = inputs.to(device), targets.to(device)

            output_images, _ = model(inputs) 
            output_images = torch.clamp(output_images, 0, 1)
            
            total_l1_loss += l1_loss_fn(output_images, targets).item()
            
            
            total_psnr += psnr(output_images, targets)
            total_ssim += ssim(output_images, targets)
    
    avg_l1_loss = total_l1_loss / len(dataloader)
    avg_psnr = total_psnr / len(dataloader)
    avg_ssim = total_ssim / len(dataloader)
    
    print(f"Evaluation - Avg L1 Loss: {avg_l1_loss:.4f}, Avg PSNR: {avg_psnr:.2f}, Avg SSIM: {avg_ssim:.4f}")
    return avg_l1_loss, avg_psnr, avg_ssim

In [7]:
data_path = 'train/train/Rain13K'
test_size = 0.2
learning_rate = 1e-4
device = "cuda" if torch.cuda.is_available() else "cpu"

input_folder = os.path.join(data_path, 'input')
target_folder = os.path.join(data_path, 'target')

input_files = sorted([os.path.join(input_folder, f) for f in os.listdir(input_folder) if f.endswith(('.png', '.jpg', '.jpeg'))])
target_files = sorted([os.path.join(target_folder, f) for f in os.listdir(target_folder) if f.endswith(('.png', '.jpg', '.jpeg'))])

train_inputs, test_inputs, train_targets, test_targets = train_test_split(
    input_files, target_files, test_size=test_size, random_state=42
)

train_dataset = RainDataset(train_inputs, train_targets)
test_dataset = RainDataset(test_inputs, test_targets)

In [ ]:
import os
from torch.utils.data import DataLoader
from torch import optim
from torch.optim import AdamW
from tqdm import tqdm
import torch

epochs = 50
batch_sizes = [32, 16, 8, 4, 1]
learning_rate = 1e-4
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

base_result_dir = "result"

if not os.path.exists(base_result_dir):
    os.makedirs(base_result_dir)

for batch_size in batch_sizes:
    print(f"Testing Batch Size: {batch_size}")
    
    batch_size_dir = os.path.join(base_result_dir, f"bs{batch_size}")
    if not os.path.exists(batch_size_dir):
        os.makedirs(batch_size_dir)
    
    log_file = os.path.join(batch_size_dir, "gridsearch_log.txt")

    with open(log_file, "w") as f:
        f.write("Batch Size, Epoch, Training Loss, Avg L1 Loss, Avg PSNR, Avg SSIM\n")
    
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    model = PReNet().to(device)
    optimizer = AdamW(model.parameters(), lr=learning_rate, betas=(0.9, 0.999))

    for epoch in range(1, epochs + 1):
        print(f"Epoch {epoch}/{epochs}")
        avg_train_loss = train(model, train_loader, optimizer, device)
        avg_l1_loss, avg_psnr, avg_ssim = evaluate(model, test_loader, device)
        
        with open(log_file, "a") as f:
            f.write(f"{batch_size}, {epoch}, {avg_train_loss:.4f}, {avg_l1_loss:.4f}, {avg_psnr:.4f}, {avg_ssim:.4f}\n")
        
        print(f"Epoch {epoch}/{epochs}, Batch Size {batch_size} => "
              f"Training Loss: {avg_train_loss:.4f}, Avg PSNR: {avg_psnr:.4f}, Avg SSIM: {avg_ssim:.4f}")
    
        if (epoch % 5 == 0) or epoch == 1:
            model_save_path = os.path.join(batch_size_dir, f"PReNet_bs{batch_size}_epoch{epoch}.pth")
            torch.save(model.state_dict(), model_save_path)
            print(f"Model saved for Batch Size: {batch_size}, Epoch: {epoch}")


Testing Batch Size: 32
Epoch 1/50




valuating: 100%|██████████| 86/86 [00:30<00:00,  2.81it/s]

Evaluation - Avg L1 Loss: 0.3567, Avg PSNR: 6.57, Avg SSIM: 0.2150
Epoch 1/50, Batch Size 32 => Training Loss: 0.1265, Avg PSNR: 6.5735, Avg SSIM: 0.2150
Model saved for Batch Size: 32, Epoch: 1
Epoch 2/50




valuating: 100%|██████████| 86/86 [00:30<00:00,  2.86it/s]

Evaluation - Avg L1 Loss: 0.3545, Avg PSNR: 6.64, Avg SSIM: 0.2154
Epoch 2/50, Batch Size 32 => Training Loss: 0.1063, Avg PSNR: 6.6429, Avg SSIM: 0.2154
Epoch 3/50


...



valuating: 100%|██████████| 86/86 [00:27<00:00,  3.12it/s]

Evaluation - Avg L1 Loss: 0.3432, Avg PSNR: 6.77, Avg SSIM: 0.2324
Epoch 8/50, Batch Size 32 => Training Loss: 0.0823, Avg PSNR: 6.7689, Avg SSIM: 0.2324
Epoch 9/50




valuating: 100%|██████████| 86/86 [00:27<00:00,  3.15it/s]

Evaluation - Avg L1 Loss: 0.3426, Avg PSNR: 6.77, Avg SSIM: 0.2328
Epoch 9/50, Batch Size 32 => Training Loss: 0.0808, Avg PSNR: 6.7742, Avg SSIM: 0.2328
Epoch 10/50


Training:  27%|██▋       | 91/343 [00:59<02:42,  1.55it/s]

In [ ]:
!zip -r file.zip /kaggle/working

In [ ]:
from IPython.display import FileLink
FileLink(r'file.zip')

In [ ]:
import os
import random
import torch
import matplotlib.pyplot as plt
from torchvision import transforms
from PIL import Image

data_path = '/kaggle/input/rain13kdataset/train/train/Rain13K'
input_folder = os.path.join(data_path, 'input')
target_folder = os.path.join(data_path, 'target')

input_files = sorted([os.path.join(input_folder, f) for f in os.listdir(input_folder) if f.endswith(('.png', '.jpg', '.jpeg'))])
target_files = sorted([os.path.join(target_folder, f) for f in os.listdir(target_folder) if f.endswith(('.png', '.jpg', '.jpeg'))])

idx = random.randint(0, len(input_files) - 1)
input_image_path = input_files[idx]
target_image_path = target_files[idx]

input_image = Image.open(input_image_path).convert("RGB")
target_image = Image.open(target_image_path).convert("RGB")

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((256, 256)),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
])
input_tensor = transform(input_image).unsqueeze(0).to(device)

model = PReNet()
model.load_state_dict(torch.load('/kaggle/working/AttUNet_bs32_epoch1.pth'))
model.to(device)

model.eval()
with torch.no_grad():
    output_tensor, _ = model(input_tensor)
    output_tensor = output_tensor.squeeze(0).cpu()

postprocess = transforms.Compose([
    transforms.Normalize(mean=[-1, -1, -1], std=[2, 2, 2]),
    transforms.ToPILImage(),
])

resize_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((256, 256)),
    transforms.ToPILImage()
])

input_image_vis = postprocess(input_tensor.squeeze(0).cpu())
output_image_vis = postprocess(output_tensor)
target_image_vis = resize_transform(target_image)

plt.figure(figsize=(12, 4))
plt.subplot(1, 3, 1)
plt.title("Input Image")
plt.imshow(input_image_vis)
plt.axis("off")

plt.subplot(1, 3, 2)
plt.title("Target Image")
plt.imshow(target_image_vis)
plt.axis("off")

plt.subplot(1, 3, 3)
plt.title("Generated Output")
plt.imshow(output_image_vis)
plt.axis("off")

plt.show()
